Testing of BioKotlin Ranges

In [36]:
//If this does not exist run from cmdline: ./gradlew shadowjar
@file:DependsOn("../build/libs/biokotlin-0.01-all.jar")

In [37]:
import biokotlin.genome.*
// import seq* as Ranges will use NucSeqRecord
import biokotlin.seq.NUC.*
import biokotlin.seq.*


In [38]:
val seq = Seq("GCAGAT")

In [47]:
val rec1 = NucSeqRecord(NucSeq("ATAACACAGAGATATATC"),"1")

In [49]:
// This creates a subset of the sequence in rec1 - just seq from positions 1 to 6
// Note the array positions are 0-based, so this should pull TAACAC
val subSettedSeq = rec1[1..6]

TAACAC


In [50]:
// create a Sequence Range (SRange) object
val sRange = rec1.range(8..12)

In [51]:
println(sRange.toString())

1:8..1:12


In [52]:
val sRangeFlanked = sRange.flankBoth(5)

In [53]:
println(sRangeFlanked)

[1:3..1:7, 1:13..1:17]


In [54]:
val sRangeFlankRight = sRange.flankRight(5)

In [55]:
println(sRangeFlankRight)

1:13..1:17


In [33]:
val dnaString = "ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC"
val dnaString2 = "ACGTGGTGAATATATATGCGCGCGTGCGTGGACGTACGTACGTACGTATCAGTCAGCTGAC"
val record1 = NucSeqRecord(NucSeq(dnaString), "Sequence 1", description = "The first sequence",
                annotations = mapOf("key1" to "value1"))
val record2 = NucSeqRecord(NucSeq(dnaString2), "Sequence 2", description = "The second sequence",
                annotations = mapOf("key1" to "value1"))

var range1 = SeqPositionRanges.of(record1,8..28)
var range2 = SeqPositionRanges.of(record2,3..19)
var range3 = SeqPositionRanges.of(SeqPosition(record1, 32),SeqPosition(record1,40))
var range4 = record2.range(25..40)

// is there a way to default to this setOf without typing entire class?
// the setOf in genome.Ranges is a java Navigable set, which does the sorting
var setRanges = overlappingSetOf(SeqPositionRangeComparator.sprComparator,range1, range4, range3, range2)

java.lang.NoClassDefFoundError: com/google/common/collect/Iterables
com.google.common.collect.ImmutableMap.copyOf(ImmutableMap.java:453)
com.google.common.collect.ImmutableMap.copyOf(ImmutableMap.java:438)
biokotlin.seq.SeqRecordKt.NucSeqRecord(SeqRecord.kt:59)
biokotlin.seq.SeqRecordKt.NucSeqRecord$default(SeqRecord.kt:57)
Line_33_jupyter.<init>(Unknown Source)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:96)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:41)
kotlin.scrip

In [33]:
for (range in setRanges) {
    println(range.toString())
    println()
}

[SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=8, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)..SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=28, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)]

[SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=32, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)..SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGAC

In [29]:
// if a Java/Kotlin non-sorted set were used above, this would sort the set by seqRecord, then lowerEndpoint site
var setRangesSorted = setRanges.toSortedSet(SeqPositionRangeComparator.sprComparator)


In [30]:
for (range in setRangesSorted) {
    println(range.toString())
    println()
}

[SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=8, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)..SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=28, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)]

[SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=32, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)..SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGAC

[SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=8, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)..SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=28, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)]

[SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=32, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)..SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGAC